9 + full dataset

In [2]:
save_path_surfix = "_10"
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Disable static KV cache (major VRAM saver for FastModel)
os.environ["UNSLOTH_DISABLE_STATIC_GENERATION"] = "1"
# Optional: if you still see high VRAM, disable Unsloth compile passes
os.environ["UNSLOTH_COMPILE_DISABLE"] = "1"
# # for debugging
# os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# os.environ['TORCH_USE_CUDA_DSA'] = "1"
from unsloth import FastModel,FastLanguageModel
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
import time
from collections import defaultdict
from openai_moderation_dataset import build_training_dataloader, build_evaluation_dataloader
from peft import PeftModel
from openai_moderation_rules import DATA3_MODERATION_COLUMN_VARIANTS, OPENAI_MODERATION_COLUMN_VARIANTS
model, tokenizer = FastModel.from_pretrained(
    # model_name = "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    # model_name = "unsloth/Qwen3-8B-Base-unsloth-bnb-4bit",
    # model_name = "unsloth/Qwen3-4B-Base-unsloth-bnb-4bit",
    model_name = "unsloth/Qwen3-1.7B-Base-unsloth-bnb-4bit",
    # model_name = "unsloth/Qwen3-1.7B-unsloth-bnb-4bit",
    # model_name = "unsloth/Qwen3-1.7B-unsloth-bnb-4bit",
    # model_name="unsloth/gemma-3-12b-pt",
    # model_name="unsloth/gemma-3-4b-pt",
    # max_seq_length = 8192, # Choose any for long context!
    # resize_model_vocab = 80999, 
    load_in_4bit = True,
)
# model.model.embed_tokens.load_state_dict({'weight':torch.load('Model/reduced_embedding.pt')})

==((====))==  Unsloth 2025.8.6: Fast Qwen3 patching. Transformers: 4.54.1. vLLM: 0.10.0.
   \\   /|    NVIDIA GeForce RTX 4090. Num GPUs = 1. Max memory: 23.635 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.31. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [2]:
use_logits = True
temperature = 1.0
sample_temperature = 0.5
batch_size = 2

In [3]:
lm_head = nn.Linear(model.lm_head.in_features, 1, bias=True)
# save the lm_head for yes and no
index = [tokenizer.encode(" No")[0], tokenizer.encode(" Yes")[0]]
print(index)
lm_head.weight.data = (model.lm_head.weight[index[1]] - model.lm_head.weight[index[0]]).unsqueeze(0)
nn.init.constant_(lm_head.bias, -8.0)
lm_head = lm_head.to('cuda')

[2308, 7414]


In [4]:
train_dataloader = build_training_dataloader("Data/data_full/balanced/data-en/train_optimized_moderation_normalized.csv", \
                                             tokenizer, OPENAI_MODERATION_COLUMN_VARIANTS,\
                                             sample_uniform_mix=0.01, return_logits=use_logits, temperature=temperature, sample_temperature=sample_temperature)
eval_dataloader = build_evaluation_dataloader("Data/Data3/data3_sample_50k_len_sorted.csv", tokenizer, DATA3_MODERATION_COLUMN_VARIANTS,\
                                              text_name="comment_text", batch_size=8)
for i,(input_ids, labels, length, metadata) in enumerate(train_dataloader):
    print(tokenizer.decode(input_ids[0]))
    print(tokenizer.decode(input_ids[1]))
    print(labels)
    print(metadata)
    if i > -1:
        break

train score columns:
harassment_score
harassment_threatening_score
hate_score
hate_threatening_score
self_harm_score
self_harm_instructions_score
self_harm_intent_score
sexual_score
sexual_minors_score
violence_score
train score columns:
severe_toxicity
obscene
threat
insult
identity_attack
sexual_explicit
You are a moderator. Check if the comment violates the rule.
Rule: Be kind, not cruel — skip the harassment
User comment: >serbia being aggressive is what directly caused wwi.austria-hungary annexed bosnia asked shit ton of demands from serbia which would cancel our teritorrial integrity. what the fuck are you talking about? .\> genocide .yall throwing this word around and you don't even know what genocide means
Does it violate?
Does the comment violate the reddit rule?
Rule: Be respectful — no harassment or abuse
Comment: >that means what exactly?.that someone didn't read the study nor understands the science nor wants to understand the science nor appreciates the insane amount of w

#### Fine-tune lm_head

In [5]:
epochs = 1
accumulation_steps = int(64 / batch_size)
lr = 1e-4
clip = 1.0
label_smoothing = 0.1

In [6]:
trainable_params = [param for param in lm_head.parameters() if param.requires_grad]
# trainable_params = [param for param in model.model.parameters() if param.requires_grad] + [lm_head_weight]
# optimizer = torch.optim.Adam([
#     {'params': [lm_head_weight], 'lr': lr},
#     {'params': [param for param in model.model.parameters() if param.requires_grad], 'lr': lr * 0.2}])
optimizer = torch.optim.Adam(trainable_params, lr=lr)
loss_fn = torch.nn.HuberLoss(delta=2.0) if use_logits else torch.nn.BCEWithLogitsLoss()
print(f"Trainable parameters: {len(trainable_params)}")

Trainable parameters: 2


In [7]:
# --- Configuration ---
logging_steps = 4000
# --- Training Loop ---
start_time = time.time()
train_loss_accum = 0
# New accumulator for more frequent logging
logging_loss_accum = 0

for epoch in range(epochs):
    # Set model to training mode    
    for i, (input_ids, labels, length, metadata) in enumerate(train_dataloader):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            input_ids, length, labels = input_ids.to('cuda'), length.to('cuda'), labels.to('cuda')
            with torch.no_grad():
                output = model.model(input_ids)
            logits = lm_head(output.last_hidden_state[torch.arange(input_ids.shape[0]), length])
            loss = loss_fn(logits.squeeze(1), labels)
            # Scale loss for gradient accumulation
            scaled_loss = loss / accumulation_steps
        
        # Accumulate loss values for logging
        train_loss_accum += loss.item()
        logging_loss_accum += loss.item()
        scaled_loss.backward()
        
        # Gradient accumulation step
        if (i + 1) % accumulation_steps == 0:
            clip_grad_norm_(trainable_params, clip)
            optimizer.step()
            optimizer.zero_grad()
            
        # --- New Logging Block ---
        # Print loss every `logging_steps`
        if (i + 1) % logging_steps == 0:
            # Calculate the average loss for the last `logging_steps`
            avg_logging_loss = logging_loss_accum / logging_steps
            print(f"Epoch: {epoch} | Step: {i+1}/{len(train_dataloader)} | Loss: {avg_logging_loss:.4f}")
            # Reset the logging accumulator
            logging_loss_accum = 0
        
        if i > logging_steps * 10:
            break
    # --- End of Epoch Summary ---
    # Calculate the average loss for the entire epoch
    epoch_avg_loss = train_loss_accum / (i + 1)
    print(f"--- End of Epoch {epoch} --- Average Loss: {epoch_avg_loss:.4f} ---")
    # Reset accumulator for the next epoch
    train_loss_accum = 0

print(f"Total training time: {(time.time() - start_time)/60:.2f} minutes")

Epoch: 0 | Step: 4000/508193 | Loss: 7.5750
Epoch: 0 | Step: 8000/508193 | Loss: 7.2550
Epoch: 0 | Step: 12000/508193 | Loss: 6.9571
Epoch: 0 | Step: 16000/508193 | Loss: 6.8773
Epoch: 0 | Step: 20000/508193 | Loss: 6.5562
Epoch: 0 | Step: 24000/508193 | Loss: 6.3760
Epoch: 0 | Step: 28000/508193 | Loss: 6.2238
Epoch: 0 | Step: 32000/508193 | Loss: 6.2716
Epoch: 0 | Step: 36000/508193 | Loss: 6.1605
Epoch: 0 | Step: 40000/508193 | Loss: 6.1337
--- End of Epoch 0 --- Average Loss: 6.6384 ---
Total training time: 14.10 minutes


#### LORA

In [8]:
epochs = 1
accumulation_steps = 64
lr = 2e-5
clip = 1.0
label_smoothing = 0.1

In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)
FastLanguageModel.for_training(model)
trainable_params = [param for param in model.parameters() if param.requires_grad] + [param for param in lm_head.parameters() if param.requires_grad]
optimizer = torch.optim.AdamW(trainable_params, lr = lr)
print(len(trainable_params))

Unsloth: Making `model.base_model.model.model` require gradients
394


In [10]:
# --- Configuration ---
logging_steps = 4000
# --- Training Loop ---
start_time = time.time()
train_loss_accum = 0
# New accumulator for more frequent logging
logging_loss_accum = 0

for epoch in range(epochs):
    # Set model to training mode    
    for i, (input_ids, labels, length, metadata) in enumerate(train_dataloader):
        with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16):
            input_ids, length, labels = input_ids.to('cuda'), length.to('cuda'), labels.to('cuda')
            output = model.base_model.model.model(input_ids)
            logits = lm_head(output.last_hidden_state[torch.arange(input_ids.shape[0]), length])
            loss = loss_fn(logits.squeeze(1), labels)
            # Scale loss for gradient accumulation
            scaled_loss = loss / accumulation_steps
        
        # Accumulate loss values for logging
        train_loss_accum += loss.item()
        logging_loss_accum += loss.item()
        scaled_loss.backward()
        
        # Gradient accumulation step
        if (i + 1) % accumulation_steps == 0:
            clip_grad_norm_(trainable_params, clip)
            optimizer.step()
            optimizer.zero_grad()
            
        # --- New Logging Block ---
        # Print loss every `logging_steps`
        if (i + 1) % logging_steps == 0:
            # Calculate the average loss for the last `logging_steps`
            avg_logging_loss = logging_loss_accum / logging_steps
            print(f"Epoch: {epoch} | Step: {i+1}/{len(train_dataloader)} | Loss: {avg_logging_loss:.4f}")
            # Reset the logging accumulator
            logging_loss_accum = 0

    # --- End of Epoch Summary ---
    # Calculate the average loss for the entire epoch
    epoch_avg_loss = train_loss_accum / len(train_dataloader)
    print(f"--- End of Epoch {epoch} --- Average Loss: {epoch_avg_loss:.4f} ---")
    # Reset accumulator for the next epoch
    train_loss_accum = 0

print(f"Total training time: {(time.time() - start_time)/60:.2f} minutes")

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch: 0 | Step: 4000/508193 | Loss: 4.3232
Epoch: 0 | Step: 8000/508193 | Loss: 2.8221
Epoch: 0 | Step: 12000/508193 | Loss: 2.3124
Epoch: 0 | Step: 16000/508193 | Loss: 2.0111
Epoch: 0 | Step: 20000/508193 | Loss: 1.7953
Epoch: 0 | Step: 24000/508193 | Loss: 1.6751
Epoch: 0 | Step: 28000/508193 | Loss: 1.6220
Epoch: 0 | Step: 32000/508193 | Loss: 1.4796
Epoch: 0 | Step: 36000/508193 | Loss: 1.4801
Epoch: 0 | Step: 40000/508193 | Loss: 1.4189
Epoch: 0 | Step: 44000/508193 | Loss: 1.3746
Epoch: 0 | Step: 48000/508193 | Loss: 1.3933
Epoch: 0 | Step: 52000/508193 | Loss: 1.3224
Epoch: 0 | Step: 56000/508193 | Loss: 1.3212
Epoch: 0 | Step: 60000/508193 | Loss: 1.2789
Epoch: 0 | Step: 64000/508193 | Loss: 1.2706
Epoch: 0 | Step: 68000/508193 | Loss: 1.2452
Epoch: 0 | Step: 72000/508193 | Loss: 1.3299
Epoch: 0 | Step: 76000/508193 | Loss: 1.2533
Epoch: 0 | Step: 80000/508193 | Loss: 1.2234
Epoch: 0 | Step: 84000/508193 | Loss: 1.1881
Epoch: 0 | Step: 88000/508193 | Loss: 1.1861
Epoch: 0 | S

In [11]:
model.save_pretrained("/home/zhenlan/Desktop/Projects/AI_Moderation/Model/merged_model4b{}".format(save_path_surfix))
torch.save(lm_head.state_dict(), "/home/zhenlan/Desktop/Projects/AI_Moderation/Model/lm_head_weight{}.pth".format(save_path_surfix))

#### evaluation

In [ ]:
# load model
# model = PeftModel.from_pretrained(model, "/home/zhenlan/Desktop/Projects/AI_Moderation/Model/merged_model4b{}".format(save_path_surfix), is_trainable=False)
# lm_head = nn.Linear(model.lm_head.in_features, 1, bias=True)
# lm_head.load_state_dict(torch.load("/home/zhenlan/Desktop/Projects/AI_Moderation/Model/lm_head_weight{}.pth".format(save_path_surfix)))
# FastLanguageModel.for_inference(model)
# model.eval()
# lm_head.to('cuda')

Training dataset

In [ ]:
train_dataloader = build_training_dataloader("Data/data_full/balanced/data-en/val_optimized_moderation.csv", \
                                             tokenizer, OPENAI_MODERATION_COLUMN_VARIANTS,\
                                             batch_size=8,
                                             sample_uniform_mix=0.01, return_logits=use_logits, temperature=temperature, sample_temperature=sample_temperature)
results = defaultdict(list)
for i, (input_ids, labels, length, metadata) in enumerate(train_dataloader):
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16), torch.no_grad():
        input_ids, length  = input_ids.to('cuda'), length.to('cuda')
        output = model.base_model.model.model(input_ids)
        yhats = lm_head(output.last_hidden_state[torch.arange(input_ids.shape[0]), length])
        for m,y,yhat in zip(metadata, labels, yhats):
            results[m].append([y.item(), yhat[0].item()])

train score columns:
harassment_score
harassment_threatening_score
hate_score
hate_threatening_score
self_harm_score
self_harm_instructions_score
self_harm_intent_score
sexual_score
sexual_minors_score
violence_score


In [15]:
from scipy import stats
import numpy as np
for category, ys in results.items():
    ys = np.array(ys)
    auc = stats.spearmanr(ys[:, 0], ys[:, 1]).statistic
    print(f"{category}: {auc:.4f}")

harassment_score: 0.9147
hate_threatening_score: 0.3970
self_harm_score: 0.8163
self_harm_intent_score: 0.8004
self_harm_instructions_score: 0.6421
sexual_minors_score: 0.7006
hate_score: 0.8408
harassment_threatening_score: 0.7571
sexual_score: 0.8988
violence_score: 0.9375


Evaluation dataset

In [12]:
results = defaultdict(list)
for i, (input_ids, labels, length, metadata) in enumerate(eval_dataloader):
    with torch.amp.autocast(device_type='cuda', dtype=torch.bfloat16), torch.no_grad():
        input_ids, length  = input_ids.to('cuda'), length.to('cuda')
        output = model.base_model.model.model(input_ids)
        yhat = lm_head(output.last_hidden_state[torch.arange(input_ids.shape[0]), length])
        for m,y,yhat in zip(metadata, labels, yhat):
            results[m].append([y.item(), yhat[0].item()])

In [13]:
# batch size 8, disable random prompt
from scipy import stats
import numpy as np
for category, ys in results.items():
    ys = np.array(ys)
    auc = stats.spearmanr(ys[:, 0], ys[:, 1]).statistic
    print(f"{category}: {auc:.4f}")

severe_toxicity: 0.2863
obscene: 0.3164
threat: 0.2331
insult: 0.5126
identity_attack: 0.3918
sexual_explicit: 0.2066
